## Simple XGBoost Example

In this notebook, we show a very simple use pattern for XGBoost.  To run this, you need to 'pip install XGBoost' into your Python environment.

author: Keith Chugg (chugg@usc.edu)

ChatGPT was used in the generation of this code.

In [1]:
import xgboost as xgb
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

## Breast Cancer Dataset

* Source: Wisconsin Diagnostic Breast Cancer (WDBC) dataset
* Task: Binary classification (malignant vs. benign breast cancer)
* Features: 30 numerical features computed from digitized images of fine needle aspirates of breast masses
* Samples: 569 instances
* Classes:
- 0 = Malignant (cancerous)
- 1 = Benign (non-cancerous)

More details:  https://scikit-learn.org/stable/datasets/toy_dataset.html#breast-cancer-dataset

In [2]:
# Load dataset
data = load_breast_cancer()
X, y = data.data, data.target

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [3]:
print(data.DESCR)

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        worst/largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 0 is Mean Radi

In [4]:
print(f'X_train: shape: {X_train.shape}')
print(f'X_test: shape: {X_test.shape}\n')

print(f'y_train: shape: {y_train.shape}')
print(f'y_test: shape: {y_test.shape}\n')

print(f'Classses:  {set(y_train)}')
print(f'Class 1 (Benign) examples in train: {np.sum(y_train)}  or {100 * np.mean(y_train) : 2.2f}% ')
print(f'Class 1 (Benign) examples in test: {np.sum(y_test)} or {100 * np.mean(y_test) : 2.2f}% ')

X_train: shape: (455, 30)
X_test: shape: (114, 30)

y_train: shape: (455,)
y_test: shape: (114,)

Classses:  {0, 1}
Class 1 (Benign) examples in train: 286  or  62.86% 
Class 1 (Benign) examples in test: 71 or  62.28% 


In [5]:
# Create and train an XGBoost classifier
model = xgb.XGBClassifier(eval_metric="logloss")
model.fit(X_train, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [6]:
# Make predictions
y_pred = model.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")


Accuracy: 0.9561


## Compare with Logistic Regression
Let's run a quick comparison using logistic regression...

In [7]:
from sklearn.linear_model import LogisticRegression

# Train a Logistic Regression model
lr_model = LogisticRegression(max_iter=10000)  # Increased iterations for convergence
lr_model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Logistic Regression Accuracy: {accuracy:.4f}")

Logistic Regression Accuracy: 0.9561


## Hyperparameter OPtimization for XGBoost Using Optuna

Let's use a package to optimize hyperparameters for XGBoost -- Optuna is one such package. Use `pip install optuna'

In [ ]:
# Define the objective function for Optuna
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 300, step=50),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 5),
        "lambda": trial.suggest_float("lambda", 1e-3, 10),
        "alpha": trial.suggest_float("alpha", 1e-3, 10),
    }

    model = xgb.XGBClassifier(**params, eval_metric="logloss") 
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring="accuracy")
    return scores.mean()

# Run hyperparameter optimization
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30)  # Run 30 trials

# Best hyperparameters
best_params = study.best_params
print("Best parameters:", best_params)

# Train final model using the best parameters
best_model = xgb.XGBClassifier(**best_params, eval_metric="logloss")
best_model.fit(X_train, y_train)

# Evaluate on the test set
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Optimized XGBoost Accuracy: {accuracy:.4f}")

[I 2025-04-02 14:38:47,453] A new study created in memory with name: no-name-0e116ccd-d709-44b6-b115-ccf40e5fe940
[I 2025-04-02 14:38:47,707] Trial 0 finished with value: 0.9494505494505494 and parameters: {'n_estimators': 100, 'max_depth': 8, 'learning_rate': 0.051235878465405, 'subsample': 0.7132503251090343, 'colsample_bytree': 0.7960725516009777, 'gamma': 1.6580832525951839, 'lambda': 4.286563858249751, 'alpha': 5.601950464810335}. Best is trial 0 with value: 0.9494505494505494.
[I 2025-04-02 14:38:48,242] Trial 1 finished with value: 0.9538461538461538 and parameters: {'n_estimators': 250, 'max_depth': 8, 'learning_rate': 0.030410767263376645, 'subsample': 0.7827594339463035, 'colsample_bytree': 0.8088866042894138, 'gamma': 0.7003784483057618, 'lambda': 8.749398441356364, 'alpha': 7.295344857788613}. Best is trial 1 with value: 0.9538461538461538.
[I 2025-04-02 14:38:48,493] Trial 2 finished with value: 0.9604395604395604 and parameters: {'n_estimators': 100, 'max_depth': 3, 'lear

Best parameters: {'n_estimators': 100, 'max_depth': 3, 'learning_rate': 0.04089630023245055, 'subsample': 0.9532169988752098, 'colsample_bytree': 0.732086961815262, 'gamma': 1.8742067744240254, 'lambda': 0.2794178361479147, 'alpha': 0.7625084715496848}
Optimized XGBoost Accuracy: 0.9561
